# Test an image 

##Extracting HOG Features of a single image

In [1]:
#importing required libraries
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import numpy as np
import joblib
import cv2
import glob
import skimage
%matplotlib inline

When we give an image path, get_hog function will return the HOG features of the image.  

In [2]:
import cv2
def hog_pred(path):
  img = imread(path)
  img = skimage.transform.resize(img, (299, 299, 3))   
        
  fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
        
        
  test_img = hog_image.reshape(1,299*299)

  hog_model = joblib.load('/kaggle/input/hand-crafted-feature-models/HOG_LR_Clf.joblib')

  pred = hog_model.predict(test_img)

  return pred

When we give the path of the test image, get_gist function will return the GIST features of the image.

In [3]:
def gist_pred(path):
        
        img = imread(path)
        #img = skimage.transform.resize(img, (299, 299, 3))   
        
        # Convert the image from BGR to RGB format
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Reshape the image to a one-dimensional array
        img = img.reshape(-1)
        
        
        # Extract the GIST features using OpenCV's calcHist function
        gist_features = cv2.calcHist([img], [0], None, [256], [0, 256])
        test_img = gist_features.reshape(gist_features.shape[1],gist_features.shape[0])

        gist_model = joblib.load('/kaggle/input/hand-crafted-feature-models/GIST_LR_Clf.joblib')

        pred = gist_model.predict(test_img)
        return pred

##pretrained models

In [4]:
def loadImages(img_path):
  
  img = imread(img_path)
  img = skimage.transform.resize(img, (224, 224, 3))        
  IMG = np.array(img,dtype=np.float64)

  test_img = IMG.reshape(1,224,224,3)
        
  return test_img

##Chexnet
 extracting features using chexnet model  on a sinlge image

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

In [6]:
def chxnet_pred(path):
  x_test = loadImages(path)
  ChexNet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3), pooling='max')

  # summarize feature map shapes
  for i in range(len(ChexNet_model.layers)):
      layer = ChexNet_model.layers[i]
      # check for conv5_ layer
      if 'conv5_block3_concat' not in layer.name:
        continue
  # redefine model to output right after the 333th (conv5_block3_concat) layer
  chex_feat_model = Model(inputs=ChexNet_model.inputs, outputs=ChexNet_model.layers[333].output)

  # feature extraction train set using ChexNet
  feat_chxnet = chex_feat_model.predict(x_test)

  chxnet_model = keras.models.load_model('/kaggle/input/saved-model-file-weights/ChexNet_feat_CLF_model.h5')

  pred= chxnet_model.predict(feat_chxnet)
  classes = np.argmax(pred,axis = 1)


  return classes


###Densenet

In [7]:
import tensorflow 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet169

from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [8]:
def dense_pred(path):
  x_test = loadImages(path)
  # Instantiate convolutional base of DenseNet 169

  dense169_model = DenseNet169(weights='imagenet', include_top=False,input_shape=(224,224, 3))
  # summarize feature map shapes
  for i in range(len(dense169_model.layers)):
      layer = dense169_model.layers[i]
  # check for conv5_ layer
      if 'conv5_block32_concat' not in layer.name:
          continue

  # redefine model to output right after the 592nd (conv5_block3_concat) layer
  dense169_feat_model = Model(inputs=dense169_model.inputs, outputs=dense169_model.layers[592].output)

  # feature extraction train set using DenseNet169
  feat = dense169_feat_model.predict(x_test)

  densenet_model = keras.models.load_model('/kaggle/input/saved-model-file-weights/densenet_feat_CLF_model.h5')

  pred= densenet_model.predict(feat)
  classes = np.argmax(pred,axis = 1)

  return classes


###inception net


In [9]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential


from tensorflow.keras.applications.inception_v3 import InceptionV3 # InceptionV3
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense,Activation,AveragePooling2D,MaxPool2D, Dropout, Conv2D, MaxPooling2D, Flatten,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
def load_inp_Images(path):
  img = cv2.imread(path)
  img = skimage.transform.resize(img, (299, 299, 3))        
  IMG = np.array(img,dtype=np.float64)
  test_img = IMG.reshape(1,299,299,3)      
        
  return test_img

In [11]:
def inp_pred(path):
  test_data = load_inp_Images(path)
  inp_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299,299,3), pooling='max')

  # summarize feature map shapes
  for i in range(len(inp_model.layers)):
      layer = inp_model.layers[i]
    # check for batch_normalization_ layer
      if 'batch_normalization_' not in layer.name:
          continue
    
  # redefine model to output right after the 305th (batch_normalization_93) layer
  incep_feat_model = Model(inputs=inp_model.inputs, outputs=inp_model.layers[305].output)

  feat_inp = incep_feat_model.predict(test_data)

  inp_model = keras.models.load_model('/kaggle/input/saved-model-file-weights/Inception_feat_CLF_model.h5')

  pred = inp_model.predict(feat_inp)
  classes = np.argmax(pred,axis = 1)

  return classes


###Ensembling

In [18]:
def ensemble_pred(path):

  hog_predicted = hog_pred(path)
  gist_predicted= gist_pred(path)
  chxnet_predicted = chxnet_pred(path)
  densenet_predicted =dense_pred(path)
  inception_predicted = inp_pred(path)


  # model_predictions is a list of predictions from multiple models
  model_predictions = [hog_predicted, gist_predicted, chxnet_predicted, densenet_predicted, inception_predicted]
 # model_predictions = [hog_pred, gist_pred, chxnet_pred, densenet_pred, inception_pred]

  # Stack the predictions along axis=0
  ensemble3_prediction = np.stack(model_predictions, axis=0)

  # Check the number of dimensions of the ensemble_prediction array
  if ensemble3_prediction.ndim == 1:
     ensemble3_prediction = ensemble3_prediction[0]
  else:
      # Take the mean of the stacked predictions along axis=0
      ensemble3_prediction = np.mean(ensemble3_prediction, axis=0)

  # Convert the average predictions back to the original format
  ensemble3_prediction = ensemble3_prediction.flatten()

  # Convert the predicted class probabilities into actual class labels
  ensemble3_prediction = np.round(ensemble3_prediction).astype(int)

  if ensemble3_prediction[0] == 1:
    print("---------------------------------------------------------------------------------")
    print("Patient with Tuberculosis")
  else:
    print("-----------------------------------------------------------------------------------")
    print("No Tuberculosis")

## Testing with an Xray of a patient with Tuberculosis

In [19]:
path = '/kaggle/input/test-image-xrays/Tuberculosis Xray.jpg'
ensemble_pred(path)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  import sys


1/1 [==============================] - 0s 56ms/step
---------------------------------------------------------------------------------
Patient with Tuberculosis


## Testing with a Normal Xray 

In [21]:
path = '/kaggle/input/test-image-xrays/Normal Xray.jpg'
ensemble_pred(path)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  import sys


1/1 [==============================] - 0s 59ms/step
-----------------------------------------------------------------------------------
Normal patient
